In [1]:
import os
os.environ["LOGLEVEL"] = "PROD"

import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.model import BlueBrainCircuitModel
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light\
    import circuit_composition, circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit


('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')


A few tests for `class CellType`.

In [2]:
from neuro_dmt.models.bluebrain.circuit.model.cell_type import CellType
from collections import OrderedDict

cell_types = (
    CellType(OrderedDict((("mtype", "L23_MC"), ("layer", "L3"),     ("etype", "bNAC")))),
    CellType(OrderedDict((("mtype", "L23_MC"), ("etype", "bNAC"),   ("layer", "L3")))),
    CellType(OrderedDict((("layer", "L3"),     ("mtype", "L23_MC"), ("etype", "bNAC")))),
    CellType(OrderedDict((("layer", "L3"),     ("etype", "bNAC"),   ("mtype", "L23_MC")))),
    CellType(OrderedDict((("etype", "bNAC"),   ("mtype", "L23_MC"), ("layer", "L3")))),
    CellType(OrderedDict((("etype", "bNAC"),   ("layer", "L3"),     ("mtype", "L23_MC")))))

expected_specifiers = frozenset(("mtype", "layer", "etype"))

for cell_type in cell_types:
    assert isinstance(cell_type.value, pd.Series)
    assert len(cell_type.value) == 3
    
assert cell_types[0].specifier == expected_specifiers, cell_types[1].specifier 
assert cell_types[1].specifier == expected_specifiers, cell_types[2].specifier 
assert cell_types[2].specifier == expected_specifiers, cell_types[3].specifier 
assert cell_types[3].specifier == expected_specifiers, cell_types[4].specifier 
assert cell_types[4].specifier == expected_specifiers, cell_types[5].specifier 
assert cell_types[5].specifier == expected_specifiers, cell_types[6].specifier 

cell_types = (
    CellType(dict((("mtype", "L23_MC"), ("layer", "L3"),     ("etype", "bNAC")))),
    CellType(dict((("mtype", "L23_MC"), ("etype", "bNAC"),   ("layer", "L3")))),
    CellType(dict((("layer", "L3"),     ("mtype", "L23_MC"), ("etype", "bNAC")))),
    CellType(dict((("layer", "L3"),     ("etype", "bNAC"),   ("mtype", "L23_MC")))),
    CellType(dict((("etype", "bNAC"),   ("mtype", "L23_MC"), ("layer", "L3")))),
    CellType(dict((("etype", "bNAC"),   ("layer", "L2"),     ("mtype", "L23_MC")))))

for cell_type in cell_types:
    assert isinstance(cell_type.value, pd.Series)
    assert len(cell_type.value) == 3
    
assert cell_types[0].specifier == expected_specifiers, cell_types[1].specifier 
assert cell_types[1].specifier == expected_specifiers, cell_types[2].specifier 
assert cell_types[2].specifier == expected_specifiers, cell_types[3].specifier 
assert cell_types[3].specifier == expected_specifiers, cell_types[4].specifier 
assert cell_types[4].specifier == expected_specifiers, cell_types[5].specifier 
assert cell_types[5].specifier == expected_specifiers, cell_types[6].specifier 

cell_types = (
    CellType(pd.Series(["L23_MC", "L3", "bNAC"], index=["mtype", "layer", "etype"])),
    CellType(pd.Series(["L23_MC", "bNAC", "L3"], index=["mtype", "etype", "layer"])),
    CellType(pd.Series(["L3", "L23_MC", "bNAC"], index=["layer", "mtype", "etype"])),
    CellType(pd.Series(["L3", "bNAC", "L23_MC"], index=["layer", "etype", "mtype"])),
    CellType(pd.Series(["bNAC", "L23_MC", "L3"], index=["etype", "mtype", "layer"])),
    CellType(pd.Series(["bNAC", "L3", "L23_MC"], index=["etype", "layer", "mtype"])))

for cell_type in cell_types:
    assert isinstance(cell_type.value, pd.Series)
    assert len(cell_type.value) == 3
    
assert cell_types[0].specifier == expected_specifiers, cell_types[1].specifier 
assert cell_types[1].specifier == expected_specifiers, cell_types[2].specifier 
assert cell_types[2].specifier == expected_specifiers, cell_types[3].specifier 
assert cell_types[3].specifier == expected_specifiers, cell_types[4].specifier 
assert cell_types[4].specifier == expected_specifiers, cell_types[5].specifier 
assert cell_types[5].specifier == expected_specifiers, cell_types[6].specifier 


pathway = CellType.pathway({"mtype": "L23_MC"}, {"mtype": "L23_MC"})

assert pathway.pre_synaptic.mtype == "L23_MC", pathway
assert pathway.post_synaptic.mtype == "L23_MC", pathway


In [3]:
print(CellType({"mtype": "L23_MC"}).value)
pd.Series(
    CellType({"mtype": "L23_MC"}).value.values,
    index=pd.MultiIndex.from_tuples([
        ("pre_synaptic", v) for v in CellType({"mtype": "L23_MC"}).value.index]))

mtype    L23_MC
dtype: object


pre_synaptic  mtype    L23_MC
dtype: object

We can define a `BlueBrainCircuitModel` instance for `MockCircuit`:

In [4]:
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
circuit_model =\
    BlueBrainCircuitModel(
        mock_circuit,
        label="BlueBrainCircuitModelMock")
conn_prob = circuit_model.connection_probability

In [5]:
#example on how to calculate pathway property
cells = circuit_model.cells
conn_prob = circuit_model.connection_probability
for pairs in circuit_model.connection_probability.get_pairs(
    pre_synaptic_cells=cells.sample(n=100).rename(columns=conn_prob._at("pre_synaptic")),
    post_synaptic_cells=cells.sample(n=100).rename(columns=conn_prob._at("post_synaptic"))):
    pass
pre_synaptic_cells = cells.sample(n=100).rename(columns=conn_prob._at("pre_synaptic"))
post_synaptic_cells = cells.sample(n=100).rename(columns=conn_prob._at("post_synaptic"))
result_one_way = pd.concat(
    [pairs["pairs"].agg(conn_prob.aggregators).transpose()
     for pairs in conn_prob.get_pairs(pre_synaptic_cells, post_synaptic_cells)],
).reset_index(
    drop=True
).rename(
    columns=conn_prob.columns
).agg("sum")



In [6]:
result_another_way = pd.concat(
    [pairs[["pairs"]]
     for pairs in conn_prob.get_pairs(pre_synaptic_cells, post_synaptic_cells)],
).reset_index(
    drop=True
).assign(
    group=0
)
print(result_another_way.shape)
summary_another_way = result_another_way\
    .groupby("group")\
    .agg(conn_prob.aggregators)\
    .pairs\
    .rename(columns=conn_prob.columns)\
    .iloc[0]

(10000, 2)


In [7]:
summary_another_way
#summary_another_way.append(
#    pd.Series({
#        ("pairs", conn_prob.phenomenon): conn_prob.definition(summary_another_way)}))

pairs_total        10000.0
pairs_connected      188.0
Name: 0, dtype: float64

In [8]:
conn_prob.get_summary(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells)

pairs_total                        10000.0000
pairs_connected                      188.0000
(pairs, connection_probability)        0.0188
dtype: float64

In [16]:
pre_type = pd.Series({"mtype": "L23_MC"})
post_type = pd.Series({"mtype": "L23_MC"})
pathway = pd.Series({
    "pre_synaptic_mtype": "L23_MC",
    "post_synaptic_mtype": "L23_MC"})
result_series = conn_prob.get_summary(
    pre_synaptic_cell_group=pre_type,
    post_synaptic_cell_group=post_type, 
    number=100)

In [17]:
result_series

pairs  pairs_total              NaN
       pairs_connected          NaN
       connection_probability   NaN
dtype: float64

In [12]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables
result_groupby = conn_prob(
    groupby=GroupByVariables(
        frozenset(["mtype"]),
        frozenset(["mtype"])))

In [13]:
result_groupby

pairs_total  pairs_connected  \
pre_synaptic_mtype post_synaptic_mtype                                 
L1_DAC             L1_DAC                         1              0.0   
                   L1_SLAC                        2              0.0   
                   L23_ChC                        2              0.0   
                   L23_SBC                        3              0.0   
                   L4_LBC                         1              0.0   
...                                             ...              ...   
L6_UPC             L6_ChC                        11              0.0   
                   L6_IPC                       132              1.0   
                   L6_TPC:A                      44              0.0   
                   L6_TPC:B                     165              3.0   
                   L6_UPC                       143              4.0   

                                        connection_probability  
pre_synaptic_mtype post_synaptic_mtype                          
L1_DAC             L1_DAC                             0.000000  
                   L1_SLAC                            0.000000  
                   L23_ChC                            0.000000  
                   L23_SBC                            0.000000  
                   L4_LBC                             0.000000  
...                                                        ...  
L6_UPC             L6_ChC                             0.000000  
                   L6_IPC                             0.007576  
                   L6_TPC:A                           0.000000  
                   L6_TPC:B                           0.018182  
                   L6_UPC                             0.027972  

[256 rows x 3 columns]

In [43]:
#pd.concat(
    #[result_groupby,
     #pd.DataFrame({("pairs", "connection_probability"): conn_prob.definition(result_groupby)})],
    #axis=1)
    
pd.DataFrame({"pairs": result_groupby.pairs.assign(connprob=conn_prob.definition(result_groupby))})
pd.DataFrame(
    [result_groupby.])

    #columns={"connection_probability": conn_prob.definition(result_groupby)})

ValueError: If using all scalar values, you must pass an index

In [9]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables
result_dataframes = conn_prob(
    pre_synaptic_cell_group=cells,
    post_synaptic_cell_group=cells,
    groupby=GroupByVariables(("mtype", ), ("mtype", )),
    number=100,
    resample=True)

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	None
            pre_synaptic_cell_group:
            	             etype  layer  morph_class   mtype      nucleus region  \
0      not-defined      1  not-defined  L1_DAC  not-defined  brain   
1      not-defined      1  not-defined  L1_DAC  not-defined  brain   
2      not-defined      1  not-defined  L1_DAC  not-defined  brain   
3      not-defined      1  not-defined  L1_DAC  not-defined  brain   
4      not-defined      1  not-defined  L1_DAC  not-defined  brain   
5      not-defined      1  not-defined  L1_DAC  not-defi

In [10]:
result_dataframes.sum()

pairs                   total        10000.000000
                        connected      199.000000
connection_probability                   6.149679
dtype: float64

In [11]:
result_no_cell_groups = conn_prob(
    groupby=GroupByVariables(("mtype", ), ("mtype", )),
    resample=True,
    number=100)

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	None
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=('mtype',), post_synaptic_cell_type_specifier=('mtype',))
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 

In [12]:
print(result_no_cell_groups.sum())
result_no_cell_groups.head()

pairs                   total        10000.000000
                        connected      201.000000
connection_probability                   5.106456
dtype: float64


pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L1_SLAC            L1_DAC                  1       0.0                    0.0
                   L1_SLAC                 1       0.0                    0.0
                   L23_ChC                 1       0.0                    0.0
                   L23_MC                  2       0.0                    0.0
                   L23_SBC                 2       0.0                    0.0

In [13]:
pre_synaptic = pd.Series({"mtype": "L6_UPC"})
post_synaptic = pd.Series({"mtype": "L6_UPC"})
pathway = CellType.pathway(pre_synaptic, post_synaptic)
result_no_cell_groups.head()

pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L1_SLAC            L1_DAC                  1       0.0                    0.0
                   L1_SLAC                 1       0.0                    0.0
                   L23_ChC                 1       0.0                    0.0
                   L23_MC                  2       0.0                    0.0
                   L23_SBC                 2       0.0                    0.0

In [14]:
levels_and_values = [
    ('_'.join(k), v) for k, v in zip(pathway.index.values, pathway.values)]
levels = tuple('_'.join(k) for k in pathway.index.values)
values = pathway.values

result_no_cell_groups.xs(
    values,
    level=levels)


pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L6_UPC             L6_UPC                150       2.0               0.013333

In [15]:
result_no_cell_groups_no_types = conn_prob(number=1000)

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	None
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu

In [16]:
result_no_cell_groups_no_types

total        1000000
connected      19473
dtype: int64

Test pathway properties in circuit analysis, using the mock circuit.

In [1]:
from neuro_dmt.models.bluebrain.circuit.adapter import BlueBrainCircuitAdapter
from neuro_dmt.models.bluebrain.circuit.test.test_connectome_analysis import *
phenomenon = Phenomenon(
    "Connection Probability",
    """
    Probability that two neurons in a pathway are connected.
    While mostly interested in `mtype->mtype` pathways, we can define
    a pathway as the group of connected neurons with pre-synaptic and
    post-synaptic neurons belonging to two parametrically defined groups.
    As an example consider neuron populations defined by not only `mtype`
    but also by soma-distance from a given location.
    """,
    group="Connectome")
pathways = mock_circuit_model.pathways(frozenset({"mtype", })).sample(n=5)
analysis = BrainCircuitAnalysis(
    phenomenon=phenomenon,
    AdapterInterface=ConnectionProbabilityInterface,
    measurement_parameters=Parameters(pathways),
        plotter=HeatMap(
        xvar=("pre_synaptic", "mtype"),
        xlabel="pre-mtype",
        yvar=("post_synaptic", "mtype"),
        ylabel="post-mtype",
        vvar=("connection_probability", "mean")))
analysis_test = BlueBrainCircuitAnalysisTest(
    analysis=analysis)
adapter = BlueBrainCircuitAdapter()
analysis_test.test_circuit_model(circuit_label)
measurement = analysis.get_measurement(mock_circuit_model, adapter, sample_size=100)
#cpdf = list(measurement.values())[0]
#cpdf.head()


('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')


In [7]:
parameters = analysis.measurement_parameters
[adapter.get_connection_probability(mock_circuit_model, **p)
 for p in parameters.for_sampling(size=1)]

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	pre_synaptic   mtype    L6_MC
		post_synaptic  mtype    L6_MC
		dtype: object
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>               

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group mtype    L1_SLAC
dtype: object
            given that {'sampling_methodology': 'random', 'resample': False, 'number': 100}
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group {'mtype': 'L1_SLAC'

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	pre_synaptic   mtype    L6_TPC:A
		post_synaptic  mtype      L4_LBC
		dtype: object
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>         

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group {'mtype': 'L6_MC'}
            given that {'sampling_methodology': 'random', 'resample': False, 'number': 100}
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group         mtype    gid
0      L1_

[0.0, 0.0, nan, nan, 0.0]

In [12]:
measurement.shape

(100, 1)

## Pathway property

A `PathwayProperty` instance is a callable -- and callable:
```
class PathwayProperty(WithFields):
    """
    Compute and store a circuit's pathway properties.
    """
    phenomenon = Field(
        """
        A label to go with the pathway property.
        """)
    circuit_model = Field(
        """
        The circuit model for which this `PathwayProperty` has been defined.
        """)
    max_length_cell_type_specification = Field(
        """
        Max length of the cell type specifier that can be stored.
        """,
        __default_value__=2)

    def __call__(self,
            pre_synaptic_cell_group,
            post_synaptic_cell_group,
            by=None,
            given=None,
            resample=False,
            sampling_methodology=SamplingMethodology.random,
            number=100):
        """
        Compute, or retrieve from the store...

        Arguments
        -------------
        pre_synaptic_cell_group:  Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        post_synaptic_cell_group: Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        by : A property of a connection ((pre_synaptic_cell, post_synaptic_cell).
             Pathway property will be computed for given values of this variable.
             If a list of variables, the computation will be conditioned on all
             the variables.
        given: A dict providing variables and their values for which the
               the pathway property will be returned.
               
        Returns
        ---------------
        pd.Series if 
        
        """
```



In [ ]:
s = circuit_model.connection_probability(
    np.array([1,2]),
    np.array([3,4]))

In [ ]:
s

In [ ]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import SamplingMethodology
pre_cell_group = pd.Series({"mtype": "L23_MC"})
post_cell_group = pd.Series({"mtype": "L23_MC"})
s = circuit_model.connection_probability(
    pre_cell_group,
    post_cell_group,
    sampling_methodology=SamplingMethodology.exhaustive)

In [ ]:
mock_circuit.cells.get({"mtype": "L23_ChC"}).head()

In [ ]:
mock_circuit.cells.get(mock_circuit.connectome.afferent_gids(140)).head()

In [ ]:
s.xs("L1_DAC", level="post_synaptic_mtype")

In [ ]:
l1dacs = mock_circuit.cells.get({"mtype": "L1_DAC"}).index.values

In [ ]:
l1dac_affs = np.concatenate(
    [mock_circuit.connectome.afferent_gids(post_gid)
     for post_gid in l1dacs],
    axis=0)

In [ ]:
l1dac_aff_mtypes = mock_circuit.cells.get(l1dac_affs, ["mtype"])

In [ ]:
l1dac_aff_mtypes.mtype.value_counts()

In [ ]:
mock_circuit.cells.get().mtype.value_counts()

In [ ]:
127890 / 70

In [ ]:
circuit_model.get_connection_probability(
    pre_cell_group, post_cell_group)

In [ ]:
def get_connection_probability(
            pre_cell_type_specifier,
            post_cell_type_specifier):
        """
        Compute connection probability once for either the entire
        circuit, or samples of afferent, or samples of  efferent,
        or samples of both efferent and afferent cells.
        """
        labels_pre_specifier =[
            "pre_synaptic_{}".format(c)
            for c in pre_cell_type_specifier]
        labels_post_specifier = [
            "post_synaptic_{}".format(c)
            for c in post_cell_type_specifier]
        cells = circuit_model.cells
        def _get_summary_connections(post_cell):
            return cells[
                list(pre_cell_type_specifier)
            ].rename(
                columns=lambda c: "pre_synaptic_{}".format(c)
            ).assign(
                number_pairs=np.in1d(
                    circuit_model.cells.index.values,
                    circuit_model.connectome.afferent_gids(post_cell.gid))
            ).groupby(
                labels_pre_specifier
            ).agg(
                ["size", "sum"]
            ).rename(
                columns={"size": "total", "sum": "connected"}
            ).assign(**{
                "post_synaptic_{}".format(p): post_cell[p] 
                for p in post_cell_type_specifier}
            ).reset_index(
            ).set_index(list(
                labels_pre_specifier + labels_post_specifier)
            )

        def _connection_probability(summary):
            """
            Compute connection probability between pairs
            """
            return summary.number_pairs.connected / summary.number_pairs.total

        return pd.concat(
            [_get_summary_connections(post_cell)
             for _, post_cell in cells.iterrows()]
        ).groupby(list(
            labels_pre_specifier + labels_post_specifier)
        ).agg(
            "sum"
        ).assign(
            connection_probability=_connection_probability )


In [ ]:
#sdirect = get_connection_probability(("mtype", ), ("mtype", ))
#sdirect.xs("L1_DAC", level="post_synaptic_mtype")

In [ ]:
l1dac_aff_mtypes.mtype.value_counts()